<a href="https://colab.research.google.com/github/d-vinha/SPBD/blob/main/lab2/SPBD_Labs_mapreduce2_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python MapReduce Exercises

##1. MrJob MapReduce Word Frequency

Using the [MrJob](https://mrjob.readthedocs.io/en/latest/) library, create a map-reduce program that counts the number of occurrences of each word, **sorting** them by frequency (the words with higher occurrence first).

Check the MrJob documentation to see how multi-step MapReduce jobs can
be implemented in the same Python class.

Note that you will need a notebook cell to install MrJob before you run your solution, as shown in this week's example notebook.

##2. Weblog Analysis

Consider a set of log files captured during a DDOS (*Distributed Denial of Service*) attack, containing information for the web accesses performed during the attack to the server.

Create a new notebook that processes the log of web entries using MrJob and map-reduce to:

1. Count the number of unique IP addresses involved in the DDOS attack.

2. For each interval of 10 seconds, provide the following information: [number of requests, average execution time, maximum time, minimum time]

3. Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).


The log files contain text lines as shown below, with TAB as the separator:

date |IP_source | status_code | operation | URL | execution time |
-|-|-|-|-|-
timestamp  | string | int | string | string| float |
2016-12-06T08:58:35.318+0000|37.139.9.11|404|GET|/codemove/TTCENCUFMH3C|0.026

<br>
The log can be downloaded from:

[https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0](https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0)

Suggestion: to start, make a copy an existing notebook and modify it.

If you really must..., you can use [dateutil.parser](https://dateutil.readthedocs.io/en/stable/parser.html) for decoding timestamps.